# National Health and Nutrition Examination Survey (NHANES)¶
https://wwwn.cdc.gov/nchs/nhanes/

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014
Foram extraídos dados do NHANES 2013-2014.

Os dados com [***este formato***](https://github.com/santanche/lab2learn/blob/master/data/nhanes/README.md) estão disponíveis em arquivos CSV (Raw) nos seguintes endereços:
- [Medications Drug](https://raw.githubusercontent.com/santanche/lab2learn/master/data/nhanes/medications-drug.csv)
- [Medications Use](https://raw.githubusercontent.com/santanche/lab2learn/master/data/nhanes/medications-use.csv)
- [Demographic Person](https://raw.githubusercontent.com/santanche/lab2learn/master/data/nhanes/demographic-person.csv)

## Tarefa 1 - Esquema Relacional
Analise os arquivos CSV e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

### Esquema Relacional:
- **Demographic-person**:

DemographicPerson(__<ins>id</ins>__, gender, age)
- **Medications-drug**:

MedicationsDrug(__<ins>code</ins>__, name)
- **Medications-use**:

MedicationsUse(__<ins>person_id</ins>__, __<ins>drug_code</ins>__, days_use)

1. CHE: person_id -> DemographicPerson
2. CHE: drug_code -> MedicationsDrug


## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
CREATE TABLE DemographicPerson (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('https://raw.githubusercontent.com/santanche/lab2learn/master/data/nhanes/demographic-person.csv');

In [3]:
SELECT * FROM DemographicPerson;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [4]:
CREATE TABLE MedicationsDrug (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(60),
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('https://raw.githubusercontent.com/santanche/lab2learn/master/data/nhanes/medications-drug.csv');

In [5]:
SELECT * FROM MedicationsDrug;

In [6]:
CREATE TABLE MedicationsUse (
  Person_id VARCHAR(5) NOT NULL,
  Drug_code VARCHAR(6) NOT NULL,
  Days_use INTEGER,
  FOREIGN KEY(Person_id)
    REFERENCES DemographicPerson(Id)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
  FOREIGN KEY(Drug_code)
    REFERENCES MedicationsDrug(Code)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('https://raw.githubusercontent.com/santanche/lab2learn/master/data/nhanes/medications-use.csv');

In [7]:
SELECT * FROM MedicationsUse;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [8]:
SELECT * FROM DemographicPerson P WHERE P.Age > 60;

b) Liste as idades das pessoas cadastradas (sem repetições)

In [9]:
SELECT DISTINCT P.Age FROM DemographicPerson P;

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [10]:
SELECT P.Id, P.Age, Mu.Drug_code, Mu.Days_use FROM DemographicPerson P, MedicationsUse Mu WHERE P.Id = Mu.Person_id;

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [11]:
SELECT P.Id, P.Age, Md.Name, Mu.Days_use FROM DemographicPerson P, MedicationsUse Mu, MedicationsDrug Md 
WHERE P.Id = Mu.Person_id AND Mu.Drug_code = Md.Code;

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [12]:
SELECT DISTINCT Md.Name FROM MedicationsDrug Md 
WHERE Md.Code IN (SELECT DISTINCT Mu.Drug_Code FROM DemographicPerson P, MedicationsUse Mu WHERE P.Id = Mu.Person_id);

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [13]:
SELECT DISTINCT Md.Name FROM MedicationsDrug Md 
WHERE Md.Code IN (SELECT DISTINCT Mu.Drug_code FROM MedicationsUse Mu WHERE Mu.Days_use > 2000);